<a href="https://colab.research.google.com/github/Yoavvvvvv/NLP/blob/main/spam_dataset_pcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2) Install the following libraries: pytorch, NLTK, spaCy

In [ ]:
#!pip install torch torchvision -U #torch library instalation

In [ ]:
#pip install nltk #nltk library instalation

In [ ]:
#!pip install --upgrade spacy #spacy library instalation

In [ ]:
#import libraries
import torch
import spacy
import pandas as pd
import nltk
import matplotlib.pyplot as plt

In [ ]:
print(torch.__version__) #print the version of torch library

2.1.0+cu121


## 3) Write a program that loads the file spam.csv (source: KAGGLE)


In [ ]:
df = pd.read_csv('/content/spam.csv', encoding='ISO-8859-1') #reading the file spam

In [ ]:
df.head() #printing the dataset for visulization

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
spam = df.drop(columns = ["Unnamed: 2","Unnamed: 3", "Unnamed: 4"]) #Cleaning the data by droping the garbage cols

In [ ]:
spam = spam.rename(columns={"v1" : "label", "v2" : "msg"}) #renaming the cols for comptability

In [ ]:
spam.head() #dataset visualization

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 4.) Compute simple statistics on the data: number of SMS message, number of spams, total word count, averaged number of words per message, 5 most frequent words in the file, number of rare words (appear only once in the file).

In [ ]:
from collections import Counter
from nltk.tokenize import word_tokenize #import word_tokenizer from nltk library
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
numberOfSMS = len(spam) #computes the amount of messages
spamCount = spam[spam['label']=="spam"].shape[0]  #counting the "spam" labeled messages

In [ ]:
#print assignment requierments
print("number of SMS message : ", numberOfSMS)
print("number of spams: ", spamCount)

number of SMS message :  5572
number of spams:  747


In [ ]:
def analize_lst_statistics(msg):
  workCount = msg.apply(lambda x: len(nltk.word_tokenize(x))) #counting the words in each message

  totalCount = workCount.sum()  #counting the total words
  print("total word count :",totalCount)  #printing the total words

  meanCount = workCount.mean()  #counting the average words
  print("Avg number of words per message :",meanCount)  #printing the average words

  tokenizedMsg = msg.apply(word_tokenize) #tokenizing the messages

  #flatten the tokens so counter func will work properly
  flattenTokens = []
  for row in tokenizedMsg:
    flattenTokens.extend(row)

  #count the most frequent words in the spam ds and print the top 5
  print("\nThe 5 most frequent words are:")
  topWordFreq = Counter(flattenTokens).most_common(5)
  for ind in range(5):
    print("the ", ind+1 ," most frequent word is: '", topWordFreq[ind][0], "'  with frequency: ",topWordFreq[ind][1]  )

analize_lst_statistics(spam["msg"]) #extracting the messages from the dataset

total word count : 104193
Avg number of words per message : 18.699389806173727

The 5 most frequent words are:
the  1  most frequent word is: ' . '  with frequency:  4886
the  2  most frequent word is: ' to '  with frequency:  2148
the  3  most frequent word is: ' I '  with frequency:  1956
the  4  most frequent word is: ' you '  with frequency:  1888
the  5  most frequent word is: ' , '  with frequency:  1871


comment: '.' count as a token and there for I have counted is a word

##5) Perform tokenization twice: by using NLTK library and by Spacy library. Write what is the difference between the results.

In [ ]:
#necessary imports for the following mission
from itertools import chain
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
NLTKtokens = []
for msg in spam['msg']:
  NLTKtokens.append(word_tokenize(msg))

NLTKtokens = list(chain.from_iterable(NLTKtokens))
print("Amount of NLTK tokens is: ",len(NLTKtokens))

nlp = spacy.load("en_core_web_sm")
spacyTokens = []
tokens = []
massages = spam["msg"]

for message in massages:
  doc = nlp(message)
  tokens = [token.text for token in doc]
  spacyTokens.append(tokens)

spacyTokens = list(chain.from_iterable(spacyTokens))
print("The amoun of spacy tokens is: ", len(spacyTokens))

Amount of NLTK tokens is:  104193
The amoun of spacy tokens is:  103533


In [ ]:
#Creating a list of NLTK tokens whos NOT IN spacy tokens
NLTKuniqueTokens = []
for token in NLTKtokens:
  if token not in spacyTokens:
    NLTKuniqueTokens.append(token)

#creating a list of spacy tokens whos NOT IN NLTK tokens
spacyUniqueTokens = []
for token in spacyTokens:
  if token not in NLTKtokens:
    spacyUniqueTokens.append(token)

#print the compare of unique values
print("NLTK tokenizer returns ", len(NLTKuniqueTokens)," that does not apears in the spacy space of tokens")
print("spacy tokenizer returns", len(spacyUniqueTokens),"that does not apears in the NLTK space of tokens")

NLTK tokenizer returns  1472  that does not apears in the spacy space of tokens
spacy tokenizer returns 2445 that does not apears in the NLTK space of tokens


In [ ]:
#extracting the messages and flatten the original messages to print flat
oringinalMassages = [msg for msg in spam["msg"]]
flattenOriginalMsg = []
for i in range(len(oringinalMassages)):
  for w in oringinalMassages[i].split():
    flattenOriginalMsg.append(w)

#printing the lists of tokent and orinal words for comparison
print("Original messages      :",flattenOriginalMsg[:100] )
print("NLTK unique tokens are :", NLTKuniqueTokens[:100])
print("unique tokens are      :", spacyUniqueTokens[:100])


Original messages      : ['Go', 'until', 'jurong', 'point,', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...', 'Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question(std', 'txt', "rate)T&C's", 'apply', "08452810075over18's", 'U', 'dun', 'say', 'so', 'early', 'hor...', 'U', 'c', 'already', 'then', 'say...', 'Nah', 'I', "don't", 'think', 'he', 'goes', 'to', 'usf,', 'he', 'lives', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back!', "I'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still?']
NLTK unique tokens are : ["'Melle", '150p/day', '16+', 'wont', 'http', '//wap', 'xxxmobilemovieclub.com', 'n=QJKGIGHJJGCBL', 'dont', 'goals/team', '4txt/Ì¼1.2

## Conclusions

After tokenizeing with both NLTK and spCy we can get the key diffrece between both NLTK and spacy tokenizers techniques:

1.   spacy gets more tokens out from the text massages than NLTK
2.   spacy gets 'http://wap', 'xxxmobilemovieclub.com?n=QJKGIGHJJGCBL' for the web adress whlie NLTK tokens it into  'http', '//wap', 'xxxmobilemovieclub.com', 'n=QJKGIGHJJGCBL'

**NLTK** uses rule-based tokenization techniques, such as regular expressions and language-specific rules, to split text into tokens.

**spaCy** uses statistical models combined with rule-based approaches to perform tokenization. It considers linguistic features and context to tokenize the text effectively.

## 6.) Perform lemmatization twice: by NLTK and by Spacy libraries. Write what is the difference between the results.

In [ ]:
nltk.download('wordnet')  #dounloading NLTK data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer  #importing nltk lemmatizer
lemmatizer = WordNetLemmatizer()

NLTKlemmas = [lemmatizer.lemmatize(word) for word in NLTKtokens]


In [ ]:
spacyLemmas = []

for msg in spam["msg"]:
  for i in nlp(msg):
    spacyLemmas.append(i.lemma_)

In [ ]:
print("Original messages:",flattenOriginalMsg[:100] )
print("NLTK lemmas:      ", NLTKlemmas[:100])
print("spacy lemmas:     ", spacyLemmas[:100])

Original messages: ['Go', 'until', 'jurong', 'point,', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...', 'Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question(std', 'txt', "rate)T&C's", 'apply', "08452810075over18's", 'U', 'dun', 'say', 'so', 'early', 'hor...', 'U', 'c', 'already', 'then', 'say...', 'Nah', 'I', "don't", 'think', 'he', 'goes', 'to', 'usf,', 'he', 'lives', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back!', "I'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still?']
NLTK lemmas:       ['Go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'Ci

In [ ]:
print("NLTK lemma :  ",flattenOriginalMsg[0],"-->",NLTKlemmas[0] )
print("apacy lemmas: ",flattenOriginalMsg[0],"-->", spacyLemmas[0] )

print("\nNLTK lemma :  ",flattenOriginalMsg[17],"-->",NLTKlemmas[20] )
print("apacy lemmas: ",flattenOriginalMsg[17],"-->", spacyLemmas[20] )

print("\nNLTK lemma :  ",flattenOriginalMsg[22],"-->",NLTKlemmas[27] )
print("apacy lemmas: ",flattenOriginalMsg[22],"-->", spacyLemmas[27] )

NLTK lemma :   Go --> Go
apacy lemmas:  Go --> go

NLTK lemma :   got --> got
apacy lemmas:  got --> get

NLTK lemma :   Joking --> Joking
apacy lemmas:  Joking --> joke


In [ ]:
print("Original messages length:",len(flattenOriginalMsg))
print("NLTK lemmas length:      ", len(NLTKlemmas))
print("spacy lemmas length:     ", len(spacyLemmas))

Original messages length: 86335
NLTK lemmas length:       104193
spacy lemmas length:      103533


From the printed outputs We can inspect some diffrences between both nltk and spact lemmatizers:



1.   NLTK outputs more lemmas then spacy
2.   Wile NLTK keeps the capital letter and ing 'Joking', spacy lower the lettr and iliminate the "ing" to 'joke'
3. NLTK 'question', '(', 'std', 'txt', 'rate', ')', 'T', '&', 'C' , turns into 'question(std', 'txt', 'rate)T&C'

**Conclusion:**
nltk and spacy lemmatizers brakes the words into a root meaning variation with a diffrent way, there for we have got slightly diffrent result for each



## 7.) Perform stemming twice (NLTK and Spacy). Write what is the difference between the results.

In [ ]:
from nltk.stem.porter import PorterStemmer #importing the porterStemmer from nltk

In [ ]:
#creating a function to lemmatize massages using NLTK library
def lemmatize_message_nltk(message):
    tokens = word_tokenize(message)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

#creating a function which do stemming to massages using nltk library
def stem_message_nltk(message):
    tokens = word_tokenize(message)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return stemmed_tokens


In [ ]:
#creating a function to easily tokenize massges with NLTK
def tokenize_message_spacy(message):
    doc = nlp(message)
    return [token.text for token in doc]

#creating a function to easily lematize massges with nltk
def lemmatize_message_spacy(message):
    doc = nlp(message)
    return [token.lemma_ for token in doc]

def spacy_token_nltk_stemming(tokens):
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return stemmed_tokens

In [ ]:
#creating a new data frame contaning nltk and spacy tokens lemmas and steme to cacluate the statistics for all of them
spam2 = spam
for row in spam:
  spam2["NLTK token"] = spam2["msg"].apply(word_tokenize)
  spam2["NLTK lemma"] = spam2["msg"].apply(lemmatize_message_nltk)
  spam2['NLTK stems'] = spam2['msg'].apply(stem_message_nltk)
  spam2['tokens_spacy'] = spam2['msg'].apply(tokenize_message_spacy)
  spam2['lemmas_spacy'] = spam2['msg'].apply(lemmatize_message_spacy)
  spam2['steme_spacy'] = spam2['msg'].apply(spacy_token_nltk_stemming)

In [ ]:
spam2.head()

,label,msg,NLTK token,NLTK lemma,NLTK stems,tokens_spacy,lemmas_spacy,steme_spacy
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point, ,, crazy, .., Avail...","[Go, until, jurong, point, ,, crazy, .., Avail...","[go, until, jurong, point, ,, crazi, .., avail...","[Go, until, jurong, point, ,, crazy, .., Avail...","[go, until, jurong, point, ,, crazy, .., avail...","[g, o, , u, n, t, i, l, , j, u, r, o, n, g, ..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar, ..., Joking, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]","[o, k, , l, a, r, ., ., ., , j, o, k, i, n, ..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, in, 2, a, wkly, comp, to, win, F...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[Free, entry, in, 2, a, wkly, comp, to, win, F...","[free, entry, in, 2, a, wkly, comp, to, win, F...","[f, r, e, e, , e, n, t, r, y, , i, n, , 2, ..."
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor, ..., U, c, alrea...","[U, dun, say, so, early, hor, ..., U, c, alrea...","[u, dun, say, so, earli, hor, ..., u, c, alrea...","[U, dun, say, so, early, hor, ..., U, c, alrea...","[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, , d, u, n, , s, a, y, , s, o, , e, a, ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, do, n't, think, he, goes, to, usf, ,,...","[Nah, I, do, n't, think, he, go, to, usf, ,, h...","[nah, i, do, n't, think, he, goe, to, usf, ,, ...","[Nah, I, do, n't, think, he, goes, to, usf, ,,...","[nah, I, do, not, think, he, go, to, usf, ,, h...","[n, a, h, , i, , d, o, n, ', t, , t, h, i, ..."


In [ ]:
def original_words_count(col):
  workCount = col.apply(lambda x: len(nltk.word_tokenize(x))) #counting the words in each message
  totalCount = workCount.sum()  #counting the total words
  return totalCount

In [ ]:
def spatial_words_count(col):
  counter = 0
  for msg in col:
    for word in msg:
      counter+=1
  return counter

In [ ]:
spatial_words_count(spam2['NLTK stems'])

104193

In [ ]:
spatial_words_count(spam2['steme_spacy'])

446422

In [ ]:
print("Original messages length:",original_words_count(spam2["msg"]))
print("NLTK stemmes length:      ", spatial_words_count(spam2['NLTK stems']))
print("spacy stemms length:     ", spatial_words_count(spam2['steme_spacy']))

Original messages length: 104193
NLTK stemmes length:       104193
spacy stemms length:      446422


In [ ]:
print("Original messages:  ",flattenOriginalMsg[:100] )
print("NLTK stemme:        ", NLTKStemme[:20])
print("spacy stemme:       ", flattenSpacySteme[:20])

Original messages:   ['Go', 'until', 'jurong', 'point,', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...', 'Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...', 'Free', 'entry', 'in', '2', 'a', 'wkly', 'comp', 'to', 'win', 'FA', 'Cup', 'final', 'tkts', '21st', 'May', '2005.', 'Text', 'FA', 'to', '87121', 'to', 'receive', 'entry', 'question(std', 'txt', "rate)T&C's", 'apply', "08452810075over18's", 'U', 'dun', 'say', 'so', 'early', 'hor...', 'U', 'c', 'already', 'then', 'say...', 'Nah', 'I', "don't", 'think', 'he', 'goes', 'to', 'usf,', 'he', 'lives', 'around', 'here', 'though', 'FreeMsg', 'Hey', 'there', 'darling', "it's", 'been', '3', "week's", 'now', 'and', 'no', 'word', 'back!', "I'd", 'like', 'some', 'fun', 'you', 'up', 'for', 'it', 'still?']
NLTK stemme:         ['go', 'until', 'jurong', 'point,', 'crazy..', 'avail', 'onli', 'in', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'cine', 'there',

In [ ]:
#print comparison beween stemming of nltk and spaCy tokens
print("NLTK stemme :  ",flattenOriginalMsg[5],"-->",NLTKStemme[5] )
print("apacy lemmas: ",flattenOriginalMsg[5],"-->", spacyStemmes[7] )

NLTK stemme :   Available --> avail
apacy lemmas:  Available --> ['as', 'per', 'your', 'request', "'", 'mell', 'mell', '(', 'oru', 'minnaminungint', 'nurungu', 'vettam', ')', "'", 'ha', 'been', 'set', 'as', 'your', 'callertun', 'for', 'all', 'caller', '.', 'press', '*', '9', 'to', 'copi', 'your', 'friend', 'callertun']


As we can implement from the prints above

1.   NLTK stemme outputs the same amount of words as the original amount wile spacy lemmas are much more
2.   The NLTK stemme transformation include capital to lower case letters, 's' 'ing','y' drop



## 8.) Repeat step 4 on the resultant text achieved after tokenization, lemmatization and stemming.

In [ ]:
spam2.head() #printing the new df contans all tokens lemmas and stemms

,label,msg,NLTK token,NLTK lemma,NLTK stems,tokens_spacy,lemmas_spacy,steme_spacy
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point, ,, crazy, .., Avail...","[Go, until, jurong, point, ,, crazy, .., Avail...","[go, until, jurong, point, ,, crazi, .., avail...","[Go, until, jurong, point, ,, crazy, .., Avail...","[go, until, jurong, point, ,, crazy, .., avail...","[g, o, , u, n, t, i, l, , j, u, r, o, n, g, ..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar, ..., Joking, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]","[Ok, lar, ..., Joking, wif, u, oni, ...]","[ok, lar, ..., joke, wif, u, oni, ...]","[o, k, , l, a, r, ., ., ., , j, o, k, i, n, ..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F...","[Free, entry, in, 2, a, wkly, comp, to, win, F...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[Free, entry, in, 2, a, wkly, comp, to, win, F...","[free, entry, in, 2, a, wkly, comp, to, win, F...","[f, r, e, e, , e, n, t, r, y, , i, n, , 2, ..."
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor, ..., U, c, alrea...","[U, dun, say, so, early, hor, ..., U, c, alrea...","[u, dun, say, so, earli, hor, ..., u, c, alrea...","[U, dun, say, so, early, hor, ..., U, c, alrea...","[u, dun, say, so, early, hor, ..., u, c, alrea...","[u, , d, u, n, , s, a, y, , s, o, , e, a, ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, do, n't, think, he, goes, to, usf, ,,...","[Nah, I, do, n't, think, he, go, to, usf, ,, h...","[nah, i, do, n't, think, he, goe, to, usf, ,, ...","[Nah, I, do, n't, think, he, goes, to, usf, ,,...","[nah, I, do, not, think, he, go, to, usf, ,, h...","[n, a, h, , i, , d, o, n, ', t, , t, h, i, ..."


In [ ]:
print("NLTK tokenized amount:", len(NLTKtokens))
print("spacy tokenized amount:", len(spacyTokens))

NLTK tokenized amount: 104193
spacy tokenized amount: 103533


In [ ]:
def analize_list_statistics(pandasSerie):
  lst = pandasSerie.tolist()
  allMsg =' '.join([' '.join(sublist) for sublist in lst])
  words = allMsg.split()
  tatalWords = len(words)
  avgWordPerMsg = tatalWords/len(lst) if lst else 0
  wordFreq = Counter(words)
  mostFreq = wordFreq.most_common(5)
  rareWords = sum(1 for word,freq in wordFreq.items() if freq == 1)
  return tatalWords, avgWordPerMsg, mostFreq, rareWords

 def count_words(message):
    return len(message)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 12)

In [ ]:
print("NLTK tokens averaged number of words per message:", spam2["NLTK token"].apply(count_words).mean())
print("NLTK Most friquent token ", Counter(spam2['NLTK token'].apply(tuple).sum()).most_common(5) )

NLTK tokens averaged number of words per message: 18.699389806173727
NLTK Most friquent token  [('.', 4886), ('to', 2148), ('I', 1956), ('you', 1888), (',', 1871)]


In [ ]:
allTokens = [token for sublist in spam2['NLTK token'] for token in sublist] # Flatten the list of lists into a single list
wordFreq = Counter(allTokens) # Count the frequency of each word
rareWords = [word for word, freq in wordFreq.items() if freq == 1] # Filter out words that appear only once

print("The rare words are:", rareWords)

The rare words are: ['jurong', 'Cine', 'amore', 'Joking', 'Tb', 'chgs', 'aids', 'patent', 'cried', 'breather', 'granted', 'fulfil', 'XXXMobileMovieClub', '//wap', 'xxxmobilemovieclub.com', 'n=QJKGIGHJJGCBL', 'ThatåÕs', 'gota', 'Macedonia', 'goals/team', 'SCOTLAND', '4txt/Ì¼1.20', 'POBOXox36504W45WQ', 'Ffffffffff', 'forced', 'packing', 'Ahhh', 'Work', 'vaguely', 'apologetic', 'fallen', 'actin', 'spoilt', 'Till', 'badly', 'cheers', 'fear', 'fainting', 'housework', 'Quick', 'cuppa', 'å£5/month', 'timings', 'watts', 'Arabian', 'steed', 'Yummy', '07732584351', 'Rodger', 'Burns', 'SEEING', 'endowed', 'hep', 'immunisation', 'Fair', 'stubborn', 'sucker', 'Hospitals', 'suckers', 'thinked', 'smarter', 'crashing', 'accomodations', '\\the', 'cave\\', 'offered', 'embarassing', 'Sptv', 'Jersey', 'Devils', 'Wings', 'Ice', 'Hockey', 'Incorrect', 'SPTV', 'Mallika', 'Sherawat', 'gauti', 'sehwag', 'burger', 'Girls', 'seekers', '\\Divorce', 'Barbie\\', 'Ken', 'Finished', 'performed', 'peoples', 'operate',

In [ ]:
NLTKTokenTatalWords, NLTKTokenAvgWordPerMsg, NLTKTokenMostFreq, NLTKTokenRareWords = analize_list_statistics(spam2['NLTK token'])

In [ ]:
print("NLTK tokenized amount:", NLTKTokenTatalWords)
print("NLTK tokens averaged number of words per message:", NLTKTokenAvgWordPerMsg)
print("NLTK tokens Most friquent token ", NLTKTokenMostFreq )
print("NLTK tokens number of rare words are:", NLTKTokenRareWords)

NLTK tokenized amount: 104193
NLTK tokens averaged number of words per message: 18.699389806173727
NLTK tokens Most friquent token  [('.', 4886), ('to', 2148), ('I', 1956), ('you', 1888), (',', 1871)]
NLTK tokens number of rare words are: 6187


In [ ]:
NLTKLemmaTatalWords, NLTKLemmaAvgWordPerMsg, NLTKLemmaMostFreq, NLTKLemmaRareWords = analize_list_statistics(spam2['NLTK lemma'])

In [ ]:
print("NLTK lemma word amount:", NLTKLemmaTatalWords)
print("NLTK lemma averaged number of words per message:", NLTKLemmaAvgWordPerMsg)
print("NLTK lemma Most friquent token ", NLTKLemmaMostFreq )
print("NLTK lemma number of rare words are:", NLTKLemmaRareWords)

NLTK lemma word amount: 104193
NLTK lemma averaged number of words per message: 18.699389806173727
NLTK lemma Most friquent token  [('.', 4886), ('to', 2148), ('I', 1956), ('you', 1888), (',', 1871)]
NLTK lemma number of rare words are: 5903


In [ ]:
NLTKstemeTatalWords, NLTKstemeAvgWordPerMsg, NLTKstemeMostFreq, NLTKstemeRareWords = analize_list_statistics(spam2['NLTK stems'])

In [ ]:
print("NLTK stemes word amount:", NLTKstemeTatalWords)
print("NLTK stemes averaged number of words per message:", NLTKstemeAvgWordPerMsg)
print("NLTK stemes Most friquent token ", NLTKstemeMostFreq )
print("NLTK stemes number of rare words are:", NLTKstemeRareWords)

NLTK stemes word amount: 104193
NLTK stemes averaged number of words per message: 18.699389806173727
NLTK stemes Most friquent token  [('.', 4886), ('i', 2900), ('to', 2241), ('you', 2228), (',', 1871)]
NLTK stemes number of rare words are: 4179


In [ ]:
spacyTokenTatalWords, spacyTokenAvgWordPerMsg, spacyTokenMostFreq, spacyTokenRareWords = analize_list_statistics(spam2['tokens_spacy'])

In [ ]:
print("spacy tokenized amount:", spacyTokenTatalWords)
print("spacy tokens averaged number of words per message:", spacyTokenAvgWordPerMsg)
print("spacy tokens Most friquent token ", spacyTokenMostFreq )
print("spacy tokens number of rare words are:", spacyTokenRareWords)

spacy tokenized amount: 103084
spacy tokens averaged number of words per message: 18.500358937544867
spacy tokens Most friquent token  [('.', 4945), ('to', 2148), ('I', 1988), ('you', 1878), (',', 1857)]
spacy tokens number of rare words are: 6272


In [ ]:
spacyLemmaTatalWords, spacyLemmaAvgWordPerMsg, spacyLemmaMostFreq, spacyLemmaRareWords = analize_list_statistics(spam2['lemmas_spacy'])

In [ ]:
print("spacy lemmas amount:", spacyLemmaTatalWords)
print("spacy lemmas averaged number of words per message:", spacyLemmaAvgWordPerMsg)
print("spacy lemmas Most friquent token ", spacyLemmaMostFreq )
print("spacy lemmas number of rare words are:", spacyLemmaRareWords)

spacy lemmas amount: 103084
spacy lemmas averaged number of words per message: 18.500358937544867
spacy lemmas Most friquent token  [('.', 4945), ('I', 3722), ('be', 3260), ('to', 2309), ('you', 2217)]
spacy lemmas number of rare words are: 5359


## 9.) Compare the results of NLTK vs. Spacy.

In [ ]:
rowsNames = ['NLTK token', 'NLTK lemma', 'NLTK stems', 'tokens_spacy', 'lemmas_spacy']
totalWrdsCount = [NLTKTokenTatalWords, NLTKLemmaTatalWords, NLTKstemeTatalWords, spacyTokenTatalWords, spacyLemmaTatalWords]
avgWrdsCount = [NLTKTokenAvgWordPerMsg, NLTKLemmaAvgWordPerMsg, NLTKstemeAvgWordPerMsg, spacyTokenAvgWordPerMsg, spacyLemmaAvgWordPerMsg]
nuberOfRareWords = [NLTKTokenRareWords, NLTKLemmaRareWords, NLTKstemeRareWords, spacyTokenRareWords, spacyLemmaRareWords]

numericalComperisonDf = pd.DataFrame({
    "indicator Name":rowsNames,
    "Total word count":totalWrdsCount,
    "avg Words Count":avgWrdsCount,
    "Number Of RareWords":nuberOfRareWords
})

In [ ]:
numericalComperisonDf

,indicator Name,Total word count,avg Words Count,Number Of RareWords
0,NLTK token,104193,18.699390,6187
1,NLTK lemma,104193,18.699390,5903
2,NLTK stems,104193,18.699390,4179
3,tokens_spacy,103084,18.500359,6272
4,lemmas_spacy,103084,18.500359,5359


The above compare table between the numerical values of the NLTK and spacy tokenization, lemmatization and stemming

In [ ]:
mostFreqComparison = pd.DataFrame({
    "NLTK token":NLTKTokenMostFreq,
    "NLTK lemma":NLTKLemmaMostFreq,
    "NLTK stems":NLTKstemeMostFreq,
    "spacy token":spacyTokenMostFreq,
    "spacy lemma":spacyLemmaMostFreq
})

In [ ]:
mostFreqComparison.head()

,NLTK token,NLTK lemma,NLTK stems,spacy token,spacy lemma
0,"(., 4886)","(., 4886)","(., 4886)","(., 4945)","(., 4945)"
1,"(to, 2148)","(to, 2148)","(i, 2900)","(to, 2148)","(I, 3722)"
2,"(I, 1956)","(I, 1956)","(to, 2241)","(I, 1988)","(be, 3260)"
3,"(you, 1888)","(you, 1888)","(you, 2228)","(you, 1878)","(to, 2309)"
4,"(,, 1871)","(,, 1871)","(,, 1871)","(,, 1857)","(you, 2217)"


From the above comparison table we can see the similarity of the most frequent word in all features and the deffrence at the second most frequent word and so on to the next frequecies

## 10.)Compare the results of NLTK before and after tokenization, lemmatization and stemming
## +
## 11.) Compare the results of Spacy before and after tokenization, lemmatization and stemming

In [ ]:
originalTatalWords, originalAvgWordPerMsg, originalMostFreq, originalRareWords = analize_list_statistics(spam["label"])

rowsNames = ['Original messages','NLTK token', 'NLTK lemma', 'NLTK stems', 'tokens_spacy', 'lemmas_spacy']
totalWrdsCount = [originalTatalWords, NLTKTokenTatalWords, NLTKLemmaTatalWords, NLTKstemeTatalWords, spacyTokenTatalWords, spacyLemmaTatalWords]
avgWrdsCount = [originalAvgWordPerMsg, NLTKTokenAvgWordPerMsg, NLTKLemmaAvgWordPerMsg, NLTKstemeAvgWordPerMsg, spacyTokenAvgWordPerMsg, spacyLemmaAvgWordPerMsg]
numberOfRareWords = [originalRareWords, NLTKTokenRareWords, NLTKLemmaRareWords, NLTKstemeRareWords, spacyTokenRareWords, spacyLemmaRareWords]

numericalComperisonDfBeforeAfter = pd.DataFrame({
    "indicator Name":rowsNames,
    "Total word count":totalWrdsCount,
    "avg Words Count":avgWrdsCount,
    "Number Of RareWords":numberOfRareWords
})


In [ ]:
numericalComperisonDfBeforeAfter

,indicator Name,Total word count,avg Words Count,Number Of RareWords
0,Original messages,17463,3.134063,0
1,NLTK token,104193,18.699390,6187
2,NLTK lemma,104193,18.699390,5903
3,NLTK stems,104193,18.699390,4179
4,tokens_spacy,103084,18.500359,6272
5,lemmas_spacy,103084,18.500359,5359


From the table above we ca can see that for statistics of the tokenizers and lemmatizares and stemmers

## 12.)  Install the library BeautifulSoup

In [ ]:
pip install beautifulsoup4 #download and install beautifulsoup4 library


In [ ]:
from bs4 import BeautifulSoup #import BeautifulSoup library

## 13.)  Choose any url page. You can choose a page of yourself in a social media (e.g., facebook, Instagram, Linkdin, Twitter), or anything else

##14.) Perform data scrapping of this url. You can learn how to do it from the following links: https://www.edureka.co/blog/web-scraping-with-python/ https://zindi.africa/learn/a-beginners-guide-to-scraping-data-from-social-media or, ask CHATGPT to program it for you.

In [ ]:
import requests

In [ ]:
url = "https://en.wikipedia.org/wiki/Neuro-linguistic_programming" #chosen url

response = requests.get(url) #sending a get request to the url

In [ ]:
#check if the url requelst was successful(status_code = 200), if so parse the url of the page
if response.status_code == 200:
  soup = BeautifulSoup(response.content, 'html.parser')

# Extract text, for example, all paragraphs
text_list = [p.text for p in soup.find_all('p')]

#  Create a DataFrame
url_text_df = pd.DataFrame(text_list, columns=['v2'])

url_text_df.head()




,v2
0,\n
1,Neuro-linguistic programming (NLP) is a pseudo...
2,NLP has been adopted by some hypnotherapists a...
3,There is no scientific evidence supporting the...
4,"According to Bandler and Grinder, NLP consists..."


## 15.)  Perform steps 5-11 on the text achieved in (14).  

In [ ]:
#define function for nltk tokenize
def nltk_tokenize(message):
    tokens = nltk.word_tokenize(message)
    return tokens

#define function for nltk lematization
def nltk_lematize(message):
    tokens = word_tokenize(message)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

#define function for nltk stemming
def nltk_stemming(message):
    tokens = word_tokenize(message)
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return stemmed_tokens

In [ ]:
#define function for spacy tokenize
def spacy_tokenize(message):
    doc = nlp(message)
    tokens = [token.text for token in doc]
    return tokens

#define function for spacy lemmatization
def spacy_lemmatize(message):
    doc = nlp(message)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

def spacy_token_nltk_stemming(tokens):
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return stemmed_tokens

In [ ]:
url_text_df['nltk_tokens'] = url_text_df['v2'].apply(nltk_tokenize)
url_text_df['spacy_tokens'] = url_text_df['v2'].apply(spacy_tokenize)

url_text_df['nltk_lemma'] = url_text_df['v2'].apply(nltk_lematize)
url_text_df['spacy_lemma'] = url_text_df['v2'].apply(spacy_lemmatize)

url_text_df['nltk_stem'] = url_text_df['v2'].apply(nltk_stemming)

In [ ]:
url_text_df['spacy_token_nltk_stem'] = url_text_df['spacy_tokens'].apply(spacy_token_nltk_stemming)

In [ ]:
url_text_df.head()

,v2,nltk_tokens,spacy_tokens,nltk_lemma,spacy_lemma,nltk_stem,spacy_token_nltk_stem
0,\n,[],[\n],[],[\n],[],[\n]
1,Neuro-linguistic programming (NLP) is a pseudo...,"[Neuro-linguistic, programming, (, NLP, ), is,...","[Neuro, -, linguistic, programming, (, NLP, ),...","[Neuro-linguistic, programming, (, NLP, ), is,...","[Neuro, -, linguistic, programming, (, NLP, ),...","[neuro-linguist, program, (, nlp, ), is, a, ps...","[neuro, -, linguist, program, (, nlp, ), is, a..."
2,NLP has been adopted by some hypnotherapists a...,"[NLP, has, been, adopted, by, some, hypnothera...","[NLP, has, been, adopted, by, some, hypnothera...","[NLP, ha, been, adopted, by, some, hypnotherap...","[NLP, have, be, adopt, by, some, hypnotherapis...","[nlp, ha, been, adopt, by, some, hypnotherapis...","[nlp, ha, been, adopt, by, some, hypnotherapis..."
3,There is no scientific evidence supporting the...,"[There, is, no, scientific, evidence, supporti...","[There, is, no, scientific, evidence, supporti...","[There, is, no, scientific, evidence, supporti...","[there, be, no, scientific, evidence, support,...","[there, is, no, scientif, evid, support, the, ...","[there, is, no, scientif, evid, support, the, ..."
4,"According to Bandler and Grinder, NLP consists...","[According, to, Bandler, and, Grinder, ,, NLP,...","[According, to, Bandler, and, Grinder, ,, NLP,...","[According, to, Bandler, and, Grinder, ,, NLP,...","[accord, to, Bandler, and, Grinder, ,, NLP, co...","[accord, to, bandler, and, grinder, ,, nlp, co...","[accord, to, bandler, and, grinder, ,, nlp, co..."


In [ ]:
#amount nltk vs spacy tokens compatison
print("nltk total tokens: ", len(url_text_df['nltk_tokens'].sum()))
print("spacy total tokens: ", len(url_text_df['spacy_tokens'].sum()))

nltk total tokens:  5189
spacy total tokens:  4942


In [ ]:
#amount of nltk vs spacy lemmatization comparison
print("original total sum of words: ", len(url_text_df['v2'].sum()))
print("nltk total lammas: ", len(url_text_df['nltk_lemma'].sum()))
print("spacy total lammas: ", len(url_text_df['spacy_lemma'].sum()))

original total sum of words:  27417
nltk total lammas:  5189
spacy total lammas:  4942


In [ ]:
#specific "word" comparison
print("Original Message:", url_text_df.loc[2, 'v2'])
print("NLTK Lemmatized Tokens:", url_text_df.loc[2, 'nltk_lemma'])
print("apcy Lemmatized Tokens:", url_text_df.loc[2, 'spacy_lemma'])

Original Message: NLP has been adopted by some hypnotherapists as well as by companies that run seminars marketed as leadership training to businesses and government agencies.[9][10]

NLTK Lemmatized Tokens: ['NLP', 'ha', 'been', 'adopted', 'by', 'some', 'hypnotherapists', 'a', 'well', 'a', 'by', 'company', 'that', 'run', 'seminar', 'marketed', 'a', 'leadership', 'training', 'to', 'business', 'and', 'government', 'agency', '.', '[', '9', ']', '[', '10', ']']
apcy Lemmatized Tokens: ['NLP', 'have', 'be', 'adopt', 'by', 'some', 'hypnotherapist', 'as', 'well', 'as', 'by', 'company', 'that', 'run', 'seminar', 'market', 'as', 'leadership', 'training', 'to', 'business', 'and', 'government', 'agencies.[9][10', ']', '\n']


**Observation:**





*   the NLTK lemmas returns has->ha , been->been, as->a
*   the spacy lemmas returns has->have , been->be, as->as



In [ ]:
#stemming amount comparison
print("Original Message:", url_text_df.loc[2, 'v2'])
print("nltk total stemms:", url_text_df.loc[2, 'nltk_stem'])
print("spacy total stemms:", url_text_df.loc[2, 'spacy_token_nltk_stem'])


Original Message: NLP has been adopted by some hypnotherapists as well as by companies that run seminars marketed as leadership training to businesses and government agencies.[9][10]

nltk total stemms: ['nlp', 'ha', 'been', 'adopt', 'by', 'some', 'hypnotherapist', 'as', 'well', 'as', 'by', 'compani', 'that', 'run', 'seminar', 'market', 'as', 'leadership', 'train', 'to', 'busi', 'and', 'govern', 'agenc', '.', '[', '9', ']', '[', '10', ']']
spacy total stemms: ['nlp', 'ha', 'been', 'adopt', 'by', 'some', 'hypnotherapist', 'as', 'well', 'as', 'by', 'compani', 'that', 'run', 'seminar', 'market', 'as', 'leadership', 'train', 'to', 'busi', 'and', 'govern', 'agencies.[9][10', ']', '\n']


In [ ]:
def analize_list_and_print(pandasSerie):
  lst = pandasSerie.tolist()
  allMsg =' '.join([' '.join(sublist) for sublist in lst])
  words = allMsg.split()
  tatalWords = len(words)
  avgWordPerMsg = tatalWords/len(lst) if lst else 0
  wordFreq = Counter(words)
  mostFreq = wordFreq.most_common(5)
  rareWords = sum(1 for word,freq in wordFreq.items() if freq == 1)
  print("Total word count:", tatalWords)
  print("Average word per message:", avgWordPerMsg)
  print("Most frequent words:", mostFreq)
  print("Number of rare words:", rareWords)


In [ ]:
print("URL NLTK stemming basic statistics:")
analize_list_and_print(url_text_df['nltk_stem'])

URL NLTK stemming basic statistics:
Total word count: 5189
Average word per message: 108.10416666666667
Most frequent words: [(',', 229), ('the', 200), ('and', 189), ('[', 186), (']', 186)]
Number of rare words: 756


In [ ]:
print("URL spacy stemming basic statistics:")
analize_list_and_print(url_text_df['spacy_token_nltk_stem'])

URL spacy stemming basic statistics:
Total word count: 4831
Average word per message: 100.64583333333333
Most frequent words: [(',', 206), ('the', 201), ('and', 191), ('of', 155), (']', 136)]
Number of rare words: 745


**NLTK steme vs spacy steme observation**


both methds returns similiar result, some diffrences might be observe:
*   NLTK: agencies.[9][10] --> 'agenc' , '.' , '[' , '9' , ']'
*   List item : agencies --> 'agencies.[9][10' , ']'
Statistic calculation shows that: NLTK returns more stemes than spacy, the most 4th most frequent word is change ([ --> of) and the number of rare words is grater in NLTK than in spacy



****

## 16.)  Download one of your whatsapp chats into *.txt file (at least 50 messages in Hebrew).  

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from spacy.lang.he import Hebrew



# Example Hebrew text
hebrew_text = "הטקסט הבא מייצג שיחת ווצאפ, שיחת ווצאפ יכולה להיות מורכב ממשפט אחד או יותר"

# Processing the text
doc = nlp(hebrew_text)

# Tokenizing the text
tokens = [token.text for token in doc]

print(tokens)

['הטקסט', 'הבא', 'מייצג', 'שיחת', 'ווצאפ', ',', 'שיחת', 'ווצאפ', 'יכולה', 'להיות', 'מורכב', 'ממשפט', 'אחד', 'או', 'יותר']


In [ ]:
def spacy_hebrew_tokenize_message(message):
    # Process the text with spaCy. This runs the entire NLP pipeline.
    doc = nlp(message)
    # Extract tokens from the doc
    tokens = [token.text for token in doc]
    return tokens

def spacy_hebrew_lemmatize_message(message):
    doc = nlp(message)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return lemmatized_tokens

In [ ]:
# Step 1: Read the text file
with open('_chat.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Step 2: Process the text (optional, depending on your needs)
# In this case, stripping newline characters from each line
lines = [line.strip() for line in lines]

# Step 3: Create the DataFrame
wha_chat_df = pd.DataFrame(lines, columns=['v2'])

print(wha_chat_df)

                                                     v2
0                                                      
1     https://cointelegraph.com/news/student-interes...
2     [3.5.2023, 16:57:18] ‪+972 58‑765‑1024‬: ‫‏‪+9...
3     [3.5.2023, 21:18:34] ~ איתמר יוסף חי: אין במקר...
4     [3.5.2023, 23:43:13] ‪+972 50‑405‑0563‬: ‫‏‪+9...
...                                                 ...
1643                                                   
1644        אנחנו מרוויחים רק אם הלקוחות שלנו מרוויחים,
1645              אז יש לנו פה אינטרס משותף לצמוח ביחד.
1646  מי שמעוניין לקחת חלק בפרוייקט שישלח לי הודעה ב...
1647  [26.2.2024, 17:53:21] ~ Alex Sayenko: ‏‏קוד הא...

[1648 rows x 1 columns]


In [ ]:
#add to df text tokanizatino with nltk and spacy
wha_chat_df['nltk_tokens'] = wha_chat_df['v2'].apply(nltk_tokenize)
wha_chat_df['spacy_tokens'] = wha_chat_df['v2'].apply(spacy_hebrew_tokenize_message)

#add to df text lemmatization with nltk and spacy
wha_chat_df['nltk_lemma'] = wha_chat_df['v2'].apply(nltk_lematize)
wha_chat_df['spacy_lemma'] = wha_chat_df['v2'].apply(spacy_hebrew_lemmatize_message)

#add to df text stemming with nltk and spacy
wha_chat_df['nltk_stem'] = wha_chat_df['v2'].apply(nltk_stemming)
wha_chat_df['spacy_token_nltk_stem'] = wha_chat_df['v2'].apply(spacy_token_nltk_stemming)


In [ ]:
wha_chat_df.head()

,v2,nltk_tokens,spacy_tokens,nltk_lemma,spacy_lemma,nltk_stem,spacy_token_nltk_stem
0,,[],[],[],[],[],[]
1,https://cointelegraph.com/news/student-interes...,"[https, :, //cointelegraph.com/news/student-in...",[https://cointelegraph.com/news/student-intere...,"[http, :, //cointelegraph.com/news/student-int...",[https://cointelegraph.com/news/student-intere...,"[http, :, //cointelegraph.com/news/student-int...","[h, t, t, p, s, :, /, /, c, o, i, n, t, e, l, ..."
2,"[3.5.2023, 16:57:18] ‪+972 58‑765‑1024‬: ‫‏‪+9...","[[, 3.5.2023, ,, 16:57:18, ], ‪+972, 58‑765‑10...","[[, 3.5.2023, ,, 16:57:18, ], ‪+972, , 58‑765...","[[, 3.5.2023, ,, 16:57:18, ], ‪+972, 58‑765‑10...","[[, 3.5.2023, ,, 16:57:18, ], ‪+972, , 58‑765...","[[, 3.5.2023, ,, 16:57:18, ], ‪+972, 58‑765‑10...","[[, 3, ., 5, ., 2, 0, 2, 3, ,, , 1, 6, :, 5, ..."
3,"[3.5.2023, 21:18:34] ~ איתמר יוסף חי: אין במקר...","[[, 3.5.2023, ,, 21:18:34, ], ~, איתמר, יוסף, ...","[[, 3.5.2023, ,, 21:18:34, ], ~, , איתמר, יוס...","[[, 3.5.2023, ,, 21:18:34, ], ~, איתמר, יוסף, ...","[[, 3.5.2023, ,, 21:18:34, ], ~, , איתמר, יוס...","[[, 3.5.2023, ,, 21:18:34, ], ~, איתמר, יוסף, ...","[[, 3, ., 5, ., 2, 0, 2, 3, ,, , 2, 1, :, 1, ..."
4,"[3.5.2023, 23:43:13] ‪+972 50‑405‑0563‬: ‫‏‪+9...","[[, 3.5.2023, ,, 23:43:13, ], ‪+972, 50‑405‑05...","[[, 3.5.2023, ,, 23:43:13, ], ‪+972, , 50‑405...","[[, 3.5.2023, ,, 23:43:13, ], ‪+972, 50‑405‑05...","[[, 3.5.2023, ,, 23:43:13, ], ‪+972, , 50‑405...","[[, 3.5.2023, ,, 23:43:13, ], ‪+972, 50‑405‑05...","[[, 3, ., 5, ., 2, 0, 2, 3, ,, , 2, 3, :, 4, ..."


**Tokanizaiont comparison**

In [ ]:
#tokens visual comparison by tokens print
print("Original Message:", wha_chat_df.loc[170, 'v2'])
print("nltk rokens:", wha_chat_df.loc[170, 'nltk_stem'])
print("spacy tokens:", wha_chat_df.loc[170, 'spacy_token_nltk_stem'])

Original Message: 
nltk rokens: []
spacy tokens: []


In [ ]:
print("Original massages statistics:\n " )
print(analize_list_and_print(wha_chat_df['v2']))
print("\n")

print("nltk tokens statistics: \n")
print( analize_list_and_print(wha_chat_df['nltk_tokens']))
print("\n")

print("spacy tokens statistics: \n")
print(analize_list_and_print(wha_chat_df['spacy_tokens']))

Original massages statistics:
 
Total word count: 112220
Average word per message: 68.09466019417475
Most frequent words: [('ו', 5788), ('י', 5394), ('ה', 5240), ('2', 3894), ('ל', 3583)]
Number of rare words: 61
None


nltk tokens statistics: 

Total word count: 25385
Average word per message: 15.403519417475728
Most frequent words: [(',', 1461), (':', 1249), ('[', 978), (']', 978), ('~', 892)]
Number of rare words: 5231
None


spacy tokens statistics: 

Total word count: 24774
Average word per message: 15.032766990291263
Most frequent words: [(',', 1461), (':', 1040), (']', 978), ('~', 892), ('[', 862)]
Number of rare words: 5164
None


**Lematization comparison**

In [ ]:
#Lemas visual comparison by text print
print("Original Message:", wha_chat_df.loc[2, 'v2'])
print("nltk Lammas:", wha_chat_df.loc[2, 'nltk_lemma'])
print("spacy Lemmas:", wha_chat_df.loc[2, 'spacy_lemma'])

Original Message: [3.5.2023, 16:57:18] ‪+972 58‑765‑1024‬: ‫‏‪+972 58‑765‑1024‬ הצטרף/ה לקבוצה באמצעות קישור ההזמנה‬
nltk Lammas: ['[', '3.5.2023', ',', '16:57:18', ']', '\u202a+972', '58‑765‑1024\u202c', ':', '\u202b\u200f\u202a+972', '58‑765‑1024\u202c', 'הצטרף/ה', 'לקבוצה', 'באמצעות', 'קישור', 'ההזמנה\u202c']
spacy Lemmas: ['[', '3.5.2023', ',', '16:57:18', ']', '\u202a+972', '\xa0', '58‑765‑1024\u202c', ':', '\u202b\u200f\u202a+972', '\xa0', '58‑765‑1024\u202c', 'הצטרף', '/', 'ה', 'לקבוצה', 'באמצעות', 'קישור', 'ההזמנה\u202c']


It's seems as the hebrew lemmatizer doesn't get the words at all

In [ ]:
doc = nlp("הטקסט הבא מייצג שיחת ווצאפ, שיחת ווצאפ יכולה להיות מורכב ממשפט אחד או יותר")
for token in doc:
  print(token.text,"-->", token.lemma_)
lemmatized_tokens = [token.lemma_ for token in doc]
print(lemmatized_tokens)

הטקסט --> הטקסט
הבא --> הבא
מייצג --> מייצג
שיחת --> שיחת
ווצאפ --> ווצאפ
, --> ,
שיחת --> שיחת
ווצאפ --> ווצאפ
יכולה --> יכולה
להיות --> להיות
מורכב --> מורכב
ממשפט --> ממשפט
אחד --> אחד
או --> או
יותר --> יותר
['הטקסט', 'הבא', 'מייצג', 'שיחת', 'ווצאפ', ',', 'שיחת', 'ווצאפ', 'יכולה', 'להיות', 'מורכב', 'ממשפט', 'אחד', 'או', 'יותר']


**Stemming Comparison**

In [ ]:
#visual text inspection
print("Original Message:", wha_chat_df.loc[20, 'v2'])
print("nltk stemme:", wha_chat_df.loc[20, 'nltk_stem'])
print("spacy Lemmas:", wha_chat_df.loc[20, 'spacy_token_nltk_stem'])

Original Message: [4.5.2023, 8:18:38] ~ Alex: כשאתה בוחר בגירסה 4 זה גם לא בלי הגבלה אם אני לא טועה
nltk stemme: ['[', '4.5.2023', ',', '8:18:38', ']', '~', 'alex', ':', 'כשאתה', 'בוחר', 'בגירסה', '4', 'זה', 'גם', 'לא', 'בלי', 'הגבלה', 'אם', 'אני', 'לא', 'טועה']
spacy Lemmas: ['[', '4', '.', '5', '.', '2', '0', '2', '3', ',', ' ', '8', ':', '1', '8', ':', '3', '8', ']', ' ', '~', '\u202f', 'a', 'l', 'e', 'x', ':', ' ', 'כ', 'ש', 'א', 'ת', 'ה', ' ', 'ב', 'ו', 'ח', 'ר', ' ', 'ב', 'ג', 'י', 'ר', 'ס', 'ה', ' ', '4', ' ', 'ז', 'ה', ' ', 'ג', 'ם', ' ', 'ל', 'א', ' ', 'ב', 'ל', 'י', ' ', 'ה', 'ג', 'ב', 'ל', 'ה', ' ', 'א', 'ם', ' ', 'א', 'נ', 'י', ' ', 'ל', 'א', ' ', 'ט', 'ו', 'ע', 'ה']


Discution

comparing the results from the statistics calculations for spacy and nltk.
As we see spacy returns less wods lemmas stemms and tokens then nltk
 And there is a difference between type and number of Frequent and rare words.

Compare the results of NLTK before and after tokenization, lemmatization and stemming.
The number of words didn't change, but the number and type of most frequent and rare ones did.

The number of nltk and spaCy words didn't change, but the number and type of most frequent and rare ones did.